## *Imports*

Install necessary packages for emotional classifer.

In [5]:
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input, Embedding, LSTM, Dropout, Lambda, Layer
from transformers import TFAutoModel, AutoTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


## *Data Preprocessing & Splitting*

In [ ]:
df = pd.read_csv("EmotionDataset/emotions_dataset.csv")

word_tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
word_tokenizer.fit_on_texts(df['sentence'])

bert_tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-fa-base-uncased")

sentences = df['sentence'].tolist()
labels = pd.factorize(df['emotion'])[0]

X_train, X_test, y_train, y_test = train_test_split(sentences, labels, test_size=0.2, random_state=42, stratify=labels)

X_train_word_tokens = word_tokenizer.texts_to_sequences(X_train)
X_test_word_tokens = word_tokenizer.texts_to_sequences(X_test)

X_train_bert_tokens = bert_tokenizer(X_train, padding="max_length", truncation=True, max_length=50, return_tensors="tf")
X_test_bert_tokens = bert_tokenizer(X_test, padding="max_length", truncation=True, max_length=50, return_tensors="tf")

with open("word_tokenizer.pkl", "wb") as f:
    pickle.dump(word_tokenizer, f)

with open("bert_tokenizer.pkl", "wb") as f:
    pickle.dump(bert_tokenizer, f)

print("Word-based tokenization:")
print("X_train shape:", len(X_train_word_tokens))
print("Y_train shape:", len(y_train))
print("X_test shape:", len(X_test_word_tokens))

print("\nBERT-based tokenization:")
print("X_train shape:", X_train_bert_tokens['input_ids'].shape)
print("X_test shape:", X_test_bert_tokens['input_ids'].shape)


Word-based tokenization:
X_train shape: 4935
X_test shape: 1234
Y_train shape: 4935

BERT-based tokenization:
X_train shape: (4935, 50)
X_test shape: (1234, 50)


## *Training High-Parameter Model (BERT-based)*

In [8]:
MODEL_NAME = "HooshvareLab/bert-fa-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
bert_model = TFAutoModel.from_pretrained(MODEL_NAME)

Some layers from the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at HooshvareLab/bert-fa-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [12]:
MAX_LEN = 50

print("X_train input_ids shape:", X_train_bert_tokens['input_ids'].shape)
print("X_train attention_mask shape:", X_train_bert_tokens['attention_mask'].shape)
print("y_train shape:", len(y_train))

X_train input_ids shape: (4935, 50)
X_train attention_mask shape: (4935, 50)
y_train shape: 4935


In [13]:
class BertLayer(Layer):
    def __init__(self, bert_model, **kwargs):
        super(BertLayer, self).__init__(**kwargs)
        self.bert_model = bert_model

    def call(self, inputs):
        input_ids, attention_mask = inputs
        bert_output = self.bert_model(input_ids, attention_mask=attention_mask)
        return bert_output.pooler_output  # Extract CLS token output

input_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(MAX_LEN,), dtype=tf.int32, name="attention_mask")

bert_output = BertLayer(bert_model)([input_ids, attention_mask])

dense = Dense(128, activation='relu')(bert_output)
output = Dense(len(set(labels)), activation='softmax')(dense)

model_parsbert = Model(inputs=[input_ids, attention_mask], outputs=output)

model_parsbert.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_parsbert.fit(
    x=[X_train_bert_tokens['input_ids'], X_train_bert_tokens['attention_mask']], 
    y=np.array(y_train),  # Ensure y_train is a NumPy array
    epochs=5, batch_size=16, 
    validation_data=([X_test_bert_tokens['input_ids'], X_test_bert_tokens['attention_mask']], np.array(y_test))
)


Epoch 1/5
 79/309 ━━━━━━━━━━━━━━━━━━━━ 1:11 313ms/step - accuracy: 0.3852 - loss: 1.9531

## *Convert BERT Model to TFLite*

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_parsbert)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_parsbert = converter.convert()

with open("Models/parsbert_emotion_model.tflite", "wb") as f:
    f.write(tflite_parsbert)

## *Training Lightweight Model (LSTM)*

In [ ]:
model_lstm = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=50),
    LSTM(32, return_sequences=True),
    LSTM(16),
    Dense(16, activation='relu'),
    Dense(len(set(labels)), activation='softmax')
])


model_lstm.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_lstm.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

## *Convert LSTM Model to TFLite*

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_lstm)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model_lstm = converter.convert()
with open("Models/lstm_emotion_model.tflite", "wb") as f:
    f.write(tflite_model_lstm)

## *Test & Evaluate Models*

In [ ]:
y_pred = np.argmax(model_lstm.predict(X_test), axis=1)
print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=df['emotion'].unique(), yticklabels=df['emotion'].unique())
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix for LSTM Model")
plt.show()